In [1]:
from qiskit.circuit.random import random_circuit 
from qiskit.primitives import Sampler 
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator 
import numpy as np

from qiskit.circuit.library.n_local.real_amplitudes import RealAmplitudes
from qiskit.circuit.library.n_local.efficient_su2 import EfficientSU2



In [2]:
nqbits = 2

circ = RealAmplitudes(nqbits, entanglement='full', reps=3, insert_barriers=False)
parameters = 4*np.pi*np.random.rand(circ.num_parameters)
circ_p1 = circ.bind_parameters(parameters)

In [11]:
from math import exp
from qiskit import *
from qiskit import Aer
import numpy as np
import matplotlib.pyplot as plt
from random import randrange

sampler = Sampler()

np.random.seed(222)
def one_shot(operator):
    
    sim = Aer.get_backend('aer_simulator')
    qc = QuantumCircuit(2)
#     unitary = [qc.h, qc.sdg, qc.id] # no! Bug #1
    unitary = [[qc.h], [qc.sdg, qc.h], [qc.id]] 
    qc.h(0)
    qc.cx(0,1)
    # new code:
    for x in unitary[operator[0]]:
        x(0)
    for x in unitary[operator[1]]:
        x(1)
    qc = qc.measure_all(inplace=False)
    result = sampler.run(qc,shots=1).result()
    # qobj = assemble(qc,shots=1)
    # result = sim.run(qobj).result().get_counts()
    print(result.quasi_dists)
    return result.quasi_dists[0]

def distance(rho):
    '''
    calculate distance of two density matrix
    '''
    return np.sqrt(np.trace(rho.conjugate().transpose().dot(rho)))

hadamard = 1/np.sqrt(2)*np.array([[1,1],[1,-1]])
s_gate = np.array([[1,0],[0,-1j]],dtype=complex)
id = np.identity(2) # don't overwrite builtins with variables!
unitary = [hadamard,np.dot(hadamard,s_gate),id]
snapshot_num = 5000 # more shots
state0 = np.array([[1,0],[0,0]])
state1 = np.array([[0,0],[0,1]])
states = [state0, state1]
record_rho = np.zeros([4,4])
for i in range(snapshot_num):
    randnum = np.random.randint(0,3,size=2)
    result = one_shot(randnum)
    
    # bit0, bit1 = [int(x) for x in list(result.keys())[0]] # assuming one shot
    bits = result[result.keys()[0]]
    bit0, bit1 = format(bits,'b').zfill(2)

    U0, U1 = unitary[randnum[0]], unitary[randnum[1]]
    # No! Bug #2: your parentheses were incorrect here
    rhohat = np.kron(3* U0.conj().T @ states[bit0] @ U0 - id , 3* U1.conj().T @ states[bit1] @ U1 - id)
    record_rho = record_rho + rhohat
    
record_rho = record_rho/snapshot_num
bell_state = np.array([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0], [0.5, 0, 0, 0.5]])

print("State distance")
print(distance(record_rho - bell_state))

# VERIFICATION
print("verify 2-local expecations converge")
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
paulis = [I, X, Y, Z]

for i in range(4):
    for j in range(4):
        twolocal_pauli = np.kron(paulis[i], paulis[j])
        expectation_true = np.trace(bell_state @ twolocal_pauli)
        expectation_rhohat = np.trace(record_rho @ twolocal_pauli)
        print((i,j), abs(expectation_true - expectation_rhohat))

[{2: 1.0}]


AttributeError: 'list' object has no attribute 'keys'

In [21]:
a,b,c = format(2,'b').zfill(3)

In [22]:
a

'0'